# **Modelo Predictivo de Averías en Aeronaves**
Optimización del Mantenimiento Preventivo

---

## **1. Introducción**
### Contexto
Las averías en aeronaves representan un desafío significativo para la industria aeronáutica, generando costos operativos elevados, retrasos y riesgos de seguridad. Este proyecto tiene como objetivo desarrollar un modelo de Machine Learning para predecir dichas averías y optimizar los recursos de mantenimiento.

### Objetivo del Proyecto
Desarrollar un modelo predictivo capaz de identificar aeronaves con alto riesgo de fallos operativos utilizando datos históricos y operativos. Esto permitirá a las aerolíneas anticiparse a las averías y mejorar la eficiencia operativa.

---

## **2. Metodología**
### **2.1 Recogida de Datos**
- **Origen de los datos:** Los datos provienen del EDA "Análisis de averías en aeronaves". Son dos datasets: histórico de vuelos desde 10/01/2021 hasta el 30/09/2023 y eventos con averías desde 01/01/2021 hasta 30/12/2023. Incluyen variables como tiempos de retraso, franjas horarias y registros de averías.
- **Volumen de datos:** Más de 200,000 registros históricos.

---

### **2.2 Preparación de Datos**

- **Pasos realizados:**
  - Limpieza de duplicados y valores faltantes.
  - En los nulos se aplica moda en Station, en la columna FlightNo se aplica la creación de número consecutivos ya que no influye.
  - Reordenamos las columnas de los dataset para la futura creación de la tabla combinada.
  - Eliminación de varias columnas en ambos dataset originales.
  - Se filtra el dataset de averías para que se ajuste a la fecha del histórico de vuelos y evitar discrepancias.
  - Cómbinamos ambas tablas mediante sqlite 3, renombramos columnas. 


#### **Tablas originales**
![Tabla histórico de averías](img/eventos_con_retrasos_dataset.png)

![Tabla histórico de vuelos](img/hist_matriculas_dataset.png)
  - Query realizada:

# Crear la tabla combinada con las columnas actualizadas
query.execute('''  
    CREATE TABLE combinada AS  
    SELECT Fecha, Numero_vuelo, Station, AC , Minutos, ATA  
    FROM histmatriculas  
    UNION ALL  
    SELECT Fecha, Numero_vuelo, Station, AC, Minutos, ATA  
    FROM eventos_retrasos  
''')  

#### **Gráfico: Distribución de vuelos con y sin avería**
![Tabla histórico de vuelos](img/dataset_final.png)

---

## **3. Análisis Exploratorio (MiniEDA)**
El análisis exploratorio (EDA) identificó patrones clave en los datos que fundamentaron las decisiones del modelo.

Dataset:

Composición:  
El dataset tiene 203,300 registros y 6 columnas con la siguiente estructura:  
* Fecha: Fecha del registro (tipo object).  
* Numero_vuelo: Número de vuelo (tipo object).  
* Station: Estación relacionada (tipo object).  
* AC: Identificación del avión (tipo object) con valores faltantes.  
* Minutos: Duración en minutos (tipo float64).  
* ATA: Tipo de avería, que será nuestra variable objetivo (object).  

Análisis preliminar:

* Valores faltantes:  
La columna AC tiene 858 valores faltantes.  
Las demás columnas no presentan valores nulos.    
* Duplicados:  
No hay registros duplicados.  
Columna ATA, tenemos 157 campos con UNK.  
* Variable numéricas:    
Solo una columna, Minutos.  
Media: 0.75 minutos, pero con un rango muy amplio (0 a 1541 minutos), hay existenciad de outliers y tenemos que revisarlos.
* Distribución de la variable objetivo ATA:    
La mayoría de las observaciones están etiquetadas con los códigos "99(sin avería)" (sumando 199,987 ocurrencias), es un desbalance significativo.  
Existen códigos de avería con muy pocos registros.  

- **Pasos realizados:**
  - Limpieza de valores faltantes y transformación de los datos, en la columna AC y valores UNK en la columna ATA, aplicaremos misma casuística que en Station antes.
  - Se identifica que el dataset es desbalanceado: 98,37 % sin avería (ATA 99).
#### **Gráfico: Distribución de ATA(tipos de avería)**
![Gráfica distribucación ATA](img/distribucion_ATA.png)

_Análisis:_   
Sin averías se le asignó el código 99. No obstante, produce un gran desbalanceo y para poder el impacto de las averías sera necesario realizar sobremuestreo en el caso de trabajar con multiclase.
Una opción y creo que será a la que enfoquemos el trabajo será en pasarlo a una bianria pero antes analizaremos un poco más el dataset.

#### **Gráfico: Averías por Estación**
![Gráfico Outliers](img/Outliers_minutos.png)

_Análisis:_  
* Se detectan varios outliers más aislados cercano a los 1600 minutos, con valores muy alejados del rango. 
* La mayoría de eventos suceden hata alrededor de los 400 minutos. 

#### **Gráfico: Averías por Estación**
![Gráfico retraso por averías](img/Minutos_por_avería.png)

_Análisis:_  
* Los outliers estan más concentrados en algunas averías. 
* No es uniforme. Al igual que los rangos son variados. 

#### **Gráfico: Averías por Estación**
![Gráfico de retraso por estación](img/Minutos_por_estación.png)

_Análisis:_  
* ALgunas estaciones tienen outliers más frecuentes que en otras. 
* Destacan las bases principales de mantenimiento PDQ y PIQ.

- Analizamos averías por franjas horarias.
- Contabilizamos número de vuelos por avión.

#### **Gráfico: Averías por Estación**
![Gráfico de vuelos por estación](img/Vuelos_por_estacion.png)

_Análisis:_  
* Hay 2 estaciones que son las que más predominan por encima del resto. 

#### **Gráfico: Averías por Estación**
![Gráfico de top 10 aviones con averías](img/Promedio_top_10_aviones.png)

_Análisis:_  
* Resalta los 10 aviones con el mayor promedio de vuelos realizados, podemos identificar cuáles están sometidos a mayor carga operativa.

#### **Gráfico: Averías por Estación**
![Gráfico total de vuelos y duración de averías](img/Promedio_top_10_aviones.png)

_Análisis:_  
* Este gráfico muestra una dispersión de los aviones, donde el eje x representa el número total de vuelos realizados y el eje y representa los minutos acumulados asociados a averías.
* Los puntos están coloreados según el tipo de avería (ATA).
* Existe una tendencia general: los aviones que realizan más vuelos tienden a acumular más minutos relacionados con averías, aunque también hay mucha dispersión.
* Algunos tipos de averías (ATA) tienen una mayor duración acumulada que otros.
* Esto sugiere que el número de vuelos podría ser un factor clave para explicar la duración de las averías.
* Hay relación entre el número de vuelos y las averías.
* Nos permite identificar patrones específicos asociados a ciertos tipos de averías (ATA).

### **Conversión tarjet a binaria**
* Si reemplazamos el valor "99" por "sin_averia" en el tarjee, ATA sería categórica, ya que representa diferentes estados o tipos de averías, incluyendo "sin_averia".  
Incluso si reducimos ATA a una variable binaria (avería/no avería), seguiría siendo categórica porque tiene dos categorías distintas.

**Decisión:**

Trabajar con multiclase nos requerirá técnicas como:
Sobremuestreo: Crear más ejemplos de la clase minoritaria.
Submuestreo: Reducir ejemplos de la clase mayoritaria.
Ajuste de pesos: Dar más importancia a las clases minoritarias durante el entrenamiento del modelo.  

En este nos decantamos a convertir la variable en binaria.  
El objetivo del proyecto es crear un modelo funcional y "rápido" para evaluar el impacto de las averías.
Sin avería 0 (ATA 99) y con avería 1 (el resto).  
  
**Pros y contras:** 
  
* *Pros*:   
 - Requiere menos cantidad de tiempo. Se intentó realizar la primera opción pero se descarto por falta del mismo.  
 - Modelo más simple y directo.  
 - Los resultados seran más sencillos de entender.  
  
* *Contras*:    
 - Se conserva menos información del tipo especifico de avería.  
 - Se podría identificar relaciones más concretas entre las variables.  

![Gráfico tarjet binaria](img/tarjet_binaria.png)

![Gráfico top 10 estaciones](img/top_10_estaciones.png)

_Análisis:_  
* Estaciones con más averías:  
Las estaciones más con retrasos son las dos bases principales donde tenemos los centros de mantenimiento.  

* Estaciones con mayor tiempo promedio de retraso (Minutos):  
Algunas estaciones, aunque no lideran en cantidad de averías, tienen tiempos de retraso considerablemente altos. Esto podría indicar problemas operativos. Además, solo aparece uno de los centros principales de mantenimiento PDQ.  

* Media de vuelos entre averías:  
Hay estaciones donde las averías son mucho más frecuentes. Esto sugiere la posibilidad de un mantenimiento ineficaz o condiciones adversas persistentes.  

* Conclusión:  
Este análisis inicial revela tendencias claras en las estaciones que generan más averías y retrasos.  
Además, la media de vuelos entre averías refuerza la importancia de investigar factores locales para mejorar la fiabilidad del servicio.  
Estas observaciones servirán para ajustar nuestro modelo predictivo.


* Creación de variables categóricas como:
  - `Franja_Horaria`: Clasificación en mañana, tarde y noche.
    - `Estación_Año`: Época del año (primavera, verano, otoño e invierno).

#### **Gráficos: Averías por franja horarios, mes y tipo de avión**
![Gráfico de averías por estación](img/Averias_clasificacion.png)

_Análisis:_  
* Averías por franja horaria:  
Las averías parecen ser más frecuente en la madrugada. Esto está relacionado con los ciclos de mantenimiento, el grueso se realiza en horario no operativo. 

* Averías por mes:  
Algunos meses presentan un mayor número de averías. Esto podría deberse a factores estacionales, como el clima o períodos de mayor actividad aérea (por ejemplo, vacaciones).  

* Averías por tipo de avión (AC):  
Hay ciertos tipos de aviones que tienen un número significativamente mayor de averías. Ciertas aeronaves podrían requerir un análisis más profundo sobre su mantenimiento o condiciones operativas.  

* Conclusión:  
Las franjas horarias y los meses con mayor incidencia de averías sugieren patrones temporales relevantes.   
Estas observaciones ofrecen bases sólidas para ajustar nuestro modelo predictivo. 

#### **Gráficos: Averías por franja horarios, mes y tipo de avión**
![Gráfico de averías por época del año](img/averias_top_aviones_por_epoca_anual.png)

_Análisis:_  
* Clustering por estaciones del año y tipo de avión:  
Los tipos de avión con más averías y las épocas del año con mayor incidencia son claramente identificables.  
Avión 2 es particularmente problemático en verano e invierno.  
Avión 31 destaca tanto en verano como en otoño.  
Avión 7 tiene un número alto de averías en primavera.  

#### **Conclusiones miniEDA:**

Este análisis sugiere que los factores estacionales y tipo de avión son importantes para predecir averías.
Cone stos resultados se podría plantear nuevas variables que combinen estacionalidad y características del avión para mejorar la precisión del modelo.

* Averías por franja horaria:  
Las averías parecen ser más frecuente en la madrugada, se ejecutan trabajos planificados durante la noche fuera del horario operativo. 
Algunos meses presentan un mayor número de averías. Esto podría deberse a factores estacionales, como el clima o períodos de mayor actividad aérea (por ejemplo, vacaciones).  

* Averías por tipo de avión (AC):  
Hay ciertos tipos de aviones que tienen un número significativamente mayor de averías. Esto sugiere que ciertas aeronaves podrían requerir un análisis más profundo sobre su mantenimiento o condiciones operativas.  

* Estaciones con más averías:  
Las estaciones más con retrasos son las dos bases principales donde tenemos los centros de mantenimiento.  

* Estaciones con mayor tiempo promedio de retraso (Minutos):  
Algunas estaciones, aunque no lideran en cantidad de averías, tienen tiempos de retraso considerablemente altos. 
Podría indicar problemas operativos. Además, solo aparece uno de los centros principales de mantenimiento PDQ.  

* Media de vuelos entre averías:  
Hay estaciones donde las averías son mucho más frecuentes. 

---

## **4. Modelado**

- **Pasos realizados:**
  - Selección de variables y tarjet.
  - Dividimos en train y test.
  - Analizamos los tipos de variables y las separamos en categóricas y numéricas.
  - Aplicamos Standarscaler para las numérica.
  - Aplicamos OneHotEncoder para las categóricas.
- **Modelos Evaluados:**
  - **Logistic Regression**
  - **Random Forest**
  - **Gradient Boosting**  

  - Bucle para iterar por cada modelo de uno en uno.
- **Preprocesamiento**
  -**Pipeline**
  - **Validación cruzada**
  - **Entramiento y evaluación**
  - **Matriz de confusión**
-
  - Se filtra el dataset de averías para que se ajuste a la fecha del histórico de vuelos y evitar discrepancias.
  - Cómbinamos ambas tablas mediante sqlite 3, renombramos columnas. 

#### **Gráficos: Averías por franja horarios, mes y tipo de avión**
![Gráfico de Matriz Logistic Regression](img/Matriz_Logistic_Regression.png)

#### **Gráficos: Averías por franja horarios, mes y tipo de avión**
![Gráfico de Matriz Random Forest](img/Matriz_Random_Forest.png)

#### **Gráficos: Averías por franja horarios, mes y tipo de avión**
![Gráfico de Matriz Gradient Boosting](img/Matriz_Gradient_Boosting.png)

### Selección del Modelo
El modelo seleccionado fue **Random Forest** por su rendimiento superior:
- **ROC-AUC:** 0.9828
- **Precisión general:** 98.3%
- Aplica mútliples árboles de decisión en subconjutnos y combina resutlado para evitar sobreajuste.   
- En comparación el gradient es más preciso pero requiere mas tiempo al ser más precio, lento y sensible a los hiperparametros


- **Pasos realizados:**
  - Entrenamos el modelo.
  - Analizamos la importancia de las variables donde destacan con diferencia "minutos". 
  - Analizamos los la precisión, recall y f1-score.
  - Detección de errores sistemáticos: análisis de falsos positivos y negativos.
  - Los resultados nos dan:
   - Falsos positivos: 0.
   - Falsos negativos: 27.
  - Train, validation y test. 70%, 15%, 15%.
  - Preprocesamiento mediante OneHotEnconding.
  - Pipeline para asegurar que no haya errores.
  - Definir hiperparámtros con GridSearch:
    - Mejores hiperparámetros: {'classifier__max_depth': 10, 'classifier__min_samples_leaf': 1, 'classifier__n_estimators': 50}
    - ROC-AUC (Validación): 0.9806
    - ROC-AUC (Test): 0.9828
  - Realizamos predicción con el modelo optimizado:
    - Falso positivo: 0.
    - Falso negativos: 16.
  - Guardamos modelo como .


#### **Conclusiones modelado:**

1. **Resultados generales:**
   - Todos los modelos presentan un excelente rendimiento con métricas muy altas, reflejadas en valores de ROC-AUC cercanos a 0.98 y precisión global casi perfecta.
   - Las principales diferencias entre los modelos son mínimas, por lo que se debe evaluar el contexto y la facilidad de interpretación para seleccionar el modelo óptimo.

---

2. **Resultados individuales:**

   - **Logistic Regression:**
     - **ROC-AUC (Validación cruzada):** 0.9840 (el más alto).
     - **Matriz de confusión:** Identifica correctamente la mayoría de las averías, aunque tiene algunos falsos negativos (FN). 
     - **Precision y Recall:**
       - Clase 1 (averías): `Recall = 0.92` significa que el modelo no detecta el 8% de las averías.
       - Este modelo es más fácil de interpretar, ideal para entender qué variables están impulsando las predicciones.

   - **Random Forest:**
     - **ROC-AUC (Validación cruzada):** 0.9830.
     - **Matriz de confusión:** Mejor recall para la clase 1 (`Recall = 0.96`), reduciendo los falsos negativos en comparación con Logistic Regression.
     - Este modelo es robusto y puede capturar relaciones complejas, aunque es menos interpretable.

   - **Gradient Boosting:**
     - **ROC-AUC (Validación cruzada):** 0.9826 (el más bajo de los tres, pero aún excelente).
     - **Matriz de confusión y Recall:** Similar a Random Forest (`Recall = 0.96`).
     - Es un modelo potente para conjuntos de datos tabulares, pero requiere más tiempo de entrenamiento.

---

3. **Matriz de confusión para los tres modelos:**
   - Clase 0 (sin averías):
     - Todos los modelos tienen un rendimiento casi perfecto, clasificando correctamente la mayoría de los casos.
   - Clase 1 (averías):
     - **Logistic Regression:** Recall más bajo (92%), lo que significa que falla en detectar más averías.
     - **Random Forest y Gradient Boosting:** Recall más alto (96%), detectando más averías, pero con mayor complejidad.

---

## **5. Prueba del modelo**
- Se probó el modelo final en el conjunto de datos de prueba. Los resultados obtenidos confirman el excelente rendimiento del modelo para la predicción de averías.
### Métricas de Clasificación:
- **Precisión general:** 100% para la clase 0 (sin avería) y 97% para la clase 1 (avería).
- **Recall:** El modelo detectó correctamente el 97% de las averías reales.
- **F1-Score:** Excelente balance entre precisión y recall, con un valor promedio ponderado de 0.99.
- **Soporte:** Las métricas se basan en un total de 30,495 observaciones (29,998 sin averías y 497 con averías).

### Reporte de Clasificación:
 - La precisión perfecta en la clase 0 asegura que el modelo no genera falsos positivos, evitando inspecciones innecesarias.
 - Aunque el recall de la clase 1 es alto, los falsos negativos (~3%) representan un área de mejora crítica, ya que implican averías no detectadas.

### Roc-Auc:
 - La curva ROC demuestra la capacidad del modelo para asignar correctamente probabilidades altas a las observaciones con averías (1) y bajas a las observaciones sin averías (0).
 - El área bajo la curva (AUC) cercano a 1 refleja un modelo con un excelente rendimiento.

---

## **6. Conclusiones**
1. **Modelo robusto y confiable:** El modelo Random Forest mostró una alta capacidad predictiva con un ROC-AUC de 0.9828. El modelo evita inspecciones innecesarias al no generar falsos positivos.
2. **Impacto de negocio:** Mejora significativa en la planificación del mantenimiento preventivo, optimizando recursos, proponiendo apertura de bases en aquellas estaciones con un gran volumen de retrasos y con vuelos recurrentes.
3. **Limitaciones:** Aunque los falsos negativos son bajos (~3%), representan un área de mejora.
4. **Escalabilidad:** Debería ampliarse el alcance del modelo, incorporando datos adicionales como condiciones climáticas o historial de mantenimiento.

---
